In [6]:
from pyspark.sql import SparkSession


spark = SparkSession.builder \
        .appName("SparkLab1") \
        .config("spark.mongodb.input.uri", "mongodb://localhost:27017/test.task5") \
        .config("spark.mongodb.output.uri", "mongodb://localhost:27017/test.task5") \
        .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.2") \
        .getOrCreate()

In [7]:
data = spark.read.parquet("orders_parquet")

In [14]:
from pyspark.sql.functions import explode, struct, sum, round, collect_list

asnw = data.groupBy("order_id","date").agg(sum("price").alias("summary price"),
        collect_list(struct("food", "price")).alias("check")) \
        .select("order_id","date", "summary price", "check")

In [15]:
asnw.show()

+--------+----------+-----------------+--------------------+
|order_id|      date|    summary price|               check|
+--------+----------+-----------------+--------------------+
|   10023|2025-02-08|            443.9|[{Khinkali, 443.90}]|
|   10075|2025-09-24|            134.9|     [{Tea, 134.90}]|
|   10076|2025-04-19|           333.33| [{Nuggets, 333.33}]|
|   10077|2025-10-09|953.5699999999999|[{Chiken bowl, 37...|
|   10086|2025-06-27|            575.4|[{White wine, 575...|
|   10095|2025-05-03|           329.13|   [{Pasta, 329.13}]|
|    1010|2025-10-14|           1664.9|[{Beef, 1530.00},...|
|   10110|2025-09-17|            560.0| [{Catfish, 560.00}]|
|   10113|2025-03-27|           613.73|[{Fresh, 169.83},...|
|   10121|2025-09-29|           482.98|     [{Pie, 482.98}]|
|   10138|2025-08-14|           555.47|[{Sandwiches set,...|
|   10166|2025-09-08|          2270.07|[{Strips, 296.17}...|
|   10171|2025-04-14|           169.83|   [{Fresh, 169.83}]|
|   10179|2025-08-29|   

In [10]:
asnw.write.format("mongo").mode("append").save()